In [1]:
import json
import os

from agent_prompts import customer_sys_prompt, support_agent_sys_prompt
from autogen import ConversableAgent
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
llm_config = {
    "config_list": [
        {
            "model": "gpt-4o-mini",
            "api_key": os.getenv("OPENAI_API_KEY"),
            "price": [0.000150, 0.000600],
            "cache_seed": None,
        }
    ]
}

In [3]:
def create_agent(
    name: str,
    sys_prompt: str,
    human_input_mode: str,
    termination_msg: str | None = None,
):
    return ConversableAgent(
        name=name,
        system_message=sys_prompt,
        is_termination_msg=lambda msg: termination_msg in msg["content"]
        if termination_msg
        else None,
        human_input_mode=human_input_mode,
        llm_config=llm_config,
    )

In [4]:
def process_queries(queries: dict[str, str | list[str]]):
    guidelines = queries["guidelines"]
    f = open("convo.txt", "a")
    for i, query in enumerate(queries["customer_queries"]):
        if i == 3:
            break
        customer_agent = create_agent(
            "Customer",
            customer_sys_prompt.format(guidelines=guidelines),
            "NEVER",
            "RESOLVED",
        )
        support_agent = create_agent(
            "Support_Agent", support_agent_sys_prompt, "NEVER", "RESOLVED"
        )
        chat_result = customer_agent.initiate_chat(support_agent, message=query)
        print(chat_result.chat_history, file=f)
        f.write("\n\n")
    f.close()


In [5]:
# with open("../query_gen/queries.jsonl", "r") as f:
#     for line in f:
#         queries = json.loads(line)
#         process_queries(queries)
#         break

In [14]:
f = open("../query_gen/queries.jsonl", 'r')
for line in f:
    f = open("convo.txt", "a")
    queries = json.loads(line)
    guidelines = open(f"/home/suchitg/amazon_help/leafdirs/{queries['title']}/t.txt", "r").read()
    if queries["tone"] != "annoyed":
        continue
    for i, query in enumerate(queries["customer_queries"]):
        # if i == 3:
        #     break
        customer_agent = create_agent(
            "Customer",
            customer_sys_prompt.format(guidelines=guidelines),
            "NEVER",
            "RESOLVED",
        )
        support_agent = create_agent(
            "Support_Agent", support_agent_sys_prompt, "NEVER", "RESOLVED"
        )
        chat_result = customer_agent.initiate_chat(support_agent, message=query)
        out_json = {
            "title": queries["title"], 
            "tone": queries["tone"],
            "style": queries["style"],
            "conversation": chat_result.chat_history
        }
        json.dump(out_json, open("./convo.jsonl", 'a'))
        open("./convo.jsonl", 'a').write('\n')
        break
    break

[autogen.oai.client: 09-24 15:52:56] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-24 15:52:56] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
Customer (to Support_Agent):

1. How am I supposed to know if I can exchange my old appliance for a new one? This should be clear already.

--------------------------------------------------------------------------------
Support_Agent (to Customer):

Hello! I completely understand your frustration, and I’m here to help. To find out if you can exchange your old appliance for a new one, it typically depends on the specific appliance and the seller’s return/exchange policy.

Could you please provide me with the name of the appliance you’re considering exchanging? That way, I can give you more precise information on your options. Thank you for your patience!

--------------------------------------